# Define Features

In [1]:
import numpy as np
import pandas as pd

In [2]:
def get_item_numeric_features(df):
    numeric_feature_names = [
        'all_rating_min_max',
        'members_min_max',
        'aired_from_min_max',
        'aired_to_min_max'
    ]

    feat_df = df[numeric_feature_names]
    return feat_df.to_numpy()

def get_user_numeric_features(df):
    numeric_feature_names = [
        'user_rating_ave_min_max',
        'user_rating_std_min_max',
        'user_aired_from_ave_min_max',
        'user_aired_to_ave_min_max'
    ]

    feat_df = df[numeric_feature_names]
    return feat_df.to_numpy()

In [3]:
def get_multihot_feature(df, feat_name):
    feat_df = df[[feat_name]]
    feat_vecs = feat_df.to_numpy()
    feat_vec = np.apply_along_axis(lambda v: v[0], 1, feat_vecs)
    return feat_vec

In [4]:
def get_label(df):
    label_df = df[['label']]
    return label_df.to_numpy()

In [5]:
def get_all_features(df):
    return (
        get_multihot_feature(df, 'genres_multihot'),
        get_multihot_feature(df, 'user_liked_genres_multihot'),
        get_item_numeric_features(df),
        get_user_numeric_features(df)
    )

# MLP

x1: item categorical feature         
x2: user categorical feature        
x3: item numeric features         
x4: user numeric features

In [6]:
import tensorflow as tf
import tensorflow.keras as keras

2022-06-25 22:24:53.249015: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-25 22:24:53.249160: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
from tensorboard.plugins.hparams import api as hp

In [8]:
HP_LAYERS = hp.HParam("layers", hp.IntInterval(2, 3))
HP_LAYER_SIZE = hp.HParam("layer_size", hp.Discrete([64, 128, 256]))
HP_LEARN_RATE = hp.HParam("learn_rate", hp.Discrete([0.001, 0.003, 0.01]))

HPARAMS = [
    HP_LAYERS,
    HP_LAYER_SIZE,
    HP_LEARN_RATE
]

METRICS = [
    hp.Metric(
        "loss",
        group="train",
        display_name="loss (train)",
    ),
    hp.Metric(
        "loss",
        group="validation",
        display_name="loss (val)",
    ),
]

In [9]:
def build_model(x1_shape, x2_shape, x3_shape, x4_shape, hparams):
    x1_input = keras.layers.Input(shape=(x1_shape,))
    x2_input = keras.layers.Input(shape=(x2_shape,))
    x3_input = keras.layers.Input(shape=(x3_shape,))
    x4_input = keras.layers.Input(shape=(x4_shape,))

    # compact embedding for x1 and x2
    compact_x1 = keras.layers.Dense(10)(x1_input)
    compact_x2 = keras.layers.Dense(10)(x2_input)

    # concat all
    merge = keras.layers.concatenate([compact_x1, compact_x2, x3_input, x4_input])

    # hidden layers
    h_input = merge
    for _ in range(hparams[HP_LAYERS]):
        h = keras.layers.Dense(hparams[HP_LAYER_SIZE], activation='relu')(h_input)
        h_input = h

    # output
    output = keras.layers.Dense(1, activation='sigmoid')(h_input)

    model = keras.models.Model(inputs=[x1_input, x2_input, x3_input, x4_input], outputs=output)

    # optimizer
    opt = keras.optimizers.Adam(learning_rate=hparams[HP_LEARN_RATE])
    model.compile(
        loss='binary_crossentropy',
        optimizer=opt,
        metrics=['accuracy']
    )

    return model

# Load dataset

In [10]:
import os

In [11]:
def data_files():
    filenames = []
    for root, dirs, files in os.walk('/home/ziyuan/PycharmProjects/gars/dataset/dnn_feat_eng'):
        for file in files:
            if file.endswith('.parquet'):
                filenames.append(os.path.join(root, file))

    return filenames

In [12]:
filesnames = data_files()

In [13]:
# 模型调优总入口
# 负责构建超参数组合，并且调用模型
def test_params():
    with tf.summary.create_file_writer('hparams').as_default():
            hp.hparams_config(hparams=HPARAMS, metrics=METRICS)

    model_id = 0
    for layers in range(HP_LAYERS.domain.min_value, HP_LAYERS.domain.max_value + 1):
        for size in HP_LAYER_SIZE.domain.values:
            for rate in HP_LEARN_RATE.domain.values:
                hparams = {
                    HP_LAYERS: layers,
                    HP_LAYER_SIZE: size,
                    HP_LEARN_RATE: rate
                }

                run_model(model_id, hparams)
                model_id += 1

# 给定某一超参数组合，构建模型并训练
def run_model(model_id, hparams):
    # build model
    model = build_model(43, 43, 4, 4, hparams)
    print(f"\nmodel id: {model_id}:")
    print({h.name: hparams[h] for h in hparams})

    # config hparam logs
    log_filename = f"{model_id}"
    for h in hparams:
        log_filename += f"_{h.name}-{hparams[h]}"

    log_dir = os.path.join("hparams", log_filename)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir,
        update_freq = 10,
        profile_batch = 0
    )
    hparams_callback = hp.KerasCallback(log_dir, hparams)

    # train model
    filenames = data_files()
    for filename in filenames[:1]:
        df = pd.read_parquet(filename)

        train_df = df

        # get features
        train_x1, train_x2, train_x3, train_x4 = get_all_features(train_df)

        # get label
        train_y = get_label(train_df)

        model.fit(
            [train_x1, train_x2, train_x3, train_x4],
            train_y,
            validation_split=0.2,
            batch_size=16,
            epochs=4,
            callbacks=[tensorboard_callback, hparams_callback]
        )

In [14]:
%rm -rf hparams
test_params()

2022-06-25 22:24:59.720469: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-25 22:24:59.720626: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-25 22:24:59.720688: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-25 22:24:59.720738: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-06-25 22:24:59.720777: W tensorflow/stream_executor/platform/default/dso_loader.cc:6


model id: 0:
{'layers': 2, 'layer_size': 64, 'learn_rate': 0.001}


2022-06-25 22:25:00.590114: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/4
1759/1759 [==============================] - 3s 1ms/step - loss: 0.5802 - accuracy: 0.6976 - val_loss: 0.5867 - val_accuracy: 0.6886
Epoch 2/4
1759/1759 [==============================] - 2s 1ms/step - loss: 0.5388 - accuracy: 0.7245 - val_loss: 0.5721 - val_accuracy: 0.7065
Epoch 3/4
1759/1759 [==============================] - 2s 1ms/step - loss: 0.5213 - accuracy: 0.7399 - val_loss: 0.5876 - val_accuracy: 0.6951
Epoch 4/4
1759/1759 [==============================] - 2s 1ms/step - loss: 0.5116 - accuracy: 0.7483 - val_loss: 0.5729 - val_accuracy: 0.7119

model id: 1:
{'layers': 2, 'layer_size': 64, 'learn_rate': 0.003}
Epoch 1/4
1759/1759 [==============================] - 2s 1ms/step - loss: 0.5746 - accuracy: 0.7003 - val_loss: 0.5942 - val_accuracy: 0.6855
Epoch 2/4
1759/1759 [==============================] - 2s 1ms/step - loss: 0.5384 - accuracy: 0.7294 - val_loss: 0.5657 - val_accuracy: 0.7156
Epoch 3/4
1759/1759 [==============================] - 2s 1ms/step - loss: 

In [1]:
%load_ext tensorboard
%tensorboard --logdir hparams

Launching TensorBoard...